In [ ]:
import pandas as pd
import requests

from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

import warnings

warnings.filterwarnings('ignore')

with open('user.txt', 'r') as file:
    STEAM_ID, API_KEY = file.read().split('\n')

GAMES_URL = 'https://api.steampowered.com/IPlayerService/GetOwnedGames/v0001?key={}'
GAMES_URL = GAMES_URL.format(API_KEY)
GAMES_URL = GAMES_URL + '&include_played_free_games=1&skip_unvetted_apps=0&include_appinfo=1&steamid={}'
GAMES_URL = GAMES_URL.format(STEAM_ID)

ACHIEV_URL = '&key=' + API_KEY + '&steamid=' + STEAM_ID
ACHIEV_URL = 'https://api.steampowered.com/ISteamUserStats/GetPlayerAchievements/v0001/?appid={}' + ACHIEV_URL

session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [ ]:
response = session.get(GAMES_URL)
result = response.json()["response"]["games"]
result = sorted(result, key=lambda g: g["playtime_forever"], reverse=True)
df = pd.json_normalize(result)

In [ ]:
for appid in df['appid']:
    response = session.get(ACHIEV_URL.format(appid))

    try:
        result = response.json()['playerstats']['achievements']
        achievements = pd.json_normalize(result)
        unlocked = len(achievements[achievements.achieved == 1])
        locked = len(achievements[achievements.achieved == 0])
        total = unlocked + locked
        perfect = 0

        if unlocked == total:
            perfect = 1

    except KeyError:
        unlocked, locked, total, perfect = 0, 0, 0, 0

    df.loc[df.appid == appid, 'achiev_unlocked'] = unlocked
    df.loc[df.appid == appid, 'achiev_locked'] = locked
    df.loc[df.appid == appid, 'achiev_total'] = total
    df.loc[df.appid == appid, 'achiev_perfect'] = perfect

In [ ]:
for i in range(20):
	df['tag_' + str(i+1)] = ''

In [ ]:
for appid in df['appid']:
    page = session.get("https://store.steampowered.com/app/" + str(appid) + "/")
    soup = BeautifulSoup(page.content)
    tags = soup.find_all("a", {"class": "app_tag"})
    
    k = 1
    for tag in tags:
        tag = tag.text.strip()
        df.loc[df.appid == appid, 'tag_' + str(k)] = tag
        k += 1